<a href="https://colab.research.google.com/github/sundar-nallalagappan/Coursera_NLP_assignments/blob/main/IMDB_Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("sairam")

sairam


Implement the IMDB use case using Bi-directional LSTM

In [3]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Dense, Flatten, GlobalAveragePooling1D, LSTM, Bidirectional, Embedding
from keras.models import Sequential

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow_datasets as tfds

# Load data

In [4]:
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
imdb

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [5]:
tokenizer_subwords = info.features['text'].encoder

In [6]:
train, test = imdb['train'], imdb['test']

for row in train:
  print(row)
  break

for row in test:
  print(row)
  break

(<tf.Tensor: shape=(163,), dtype=int64, numpy=
array([  62,   18,   41,  604,  927,   65,    3,  644, 7968,   21,   35,
       5096,   36,   11,   43, 2948, 5240,  102,   50,  681, 7862, 1244,
          3, 3266,   29,  122,  640,    2,   26,   14,  279,  438,   35,
         79,  349,  384,   11, 1991,    3,  492,   79,  122,  188,  117,
         33, 4047, 4531,   14,   65, 7968,    8, 1819, 3947,    3,   62,
         27,    9,   41,  577, 5044, 2629, 2552, 7193, 7961, 3642,    3,
         19,  107, 3903,  225,   85,  198,   72,    1, 1512,  738, 2347,
        102, 6245,    8,   85,  308,   79, 6936, 7961,   23, 4981, 8044,
          3, 6429, 7961, 1141, 1335, 1848, 4848,   55, 3601, 4217, 8050,
          2,    5,   59, 3831, 1484, 8040, 7974,  174, 5773,   22, 5240,
        102,   18,  247,   26,    4, 3903, 1612, 3902,  291,   11,    4,
         27,   13,   18, 4092, 4008, 7961,    6,  119,  213, 2774,    3,
         12,  258, 2306,   13,   91,   29,  171,   52,  229,    2, 1245,
    

In [7]:
for row in train:
  print(tokenizer_subwords.decode(row[0]))
  break

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.


* padded_batch function - will padd the sequences and also put them in requested batches

In [8]:
BUFFER_SIZE = 1000
BATCH_SIZE  = 256

train_dataset = train.shuffle(BUFFER_SIZE)

train_dataset = train.padded_batch(BATCH_SIZE)
test_dataset  = test.padded_batch(BATCH_SIZE)

In [9]:
for idx, i in enumerate(train_dataset):
  print(idx)
  if idx < 4:
    print(i[0])
    print(i[0].shape)
  else:
    break

0
tf.Tensor(
[[  62   18   41 ...    0    0    0]
 [  12   31   93 ...    0    0    0]
 [ 636  102 4714 ...    0    0    0]
 ...
 [  19   77  457 ...    0    0    0]
 [5440  131 5617 ...    0    0    0]
 [3475  456  143 ...    0    0    0]], shape=(256, 1179), dtype=int64)
(256, 1179)
1
tf.Tensor(
[[  19   27   18 ...    0    0    0]
 [7914    4 3572 ...    0    0    0]
 [ 133 2690 1083 ...    0    0    0]
 ...
 [1987    2   14 ...    0    0    0]
 [ 156   37  239 ...    0    0    0]
 [1097 3837 7961 ...    0    0    0]], shape=(256, 1615), dtype=int64)
(256, 1615)
2
tf.Tensor(
[[  19   32 1422 ...    0    0    0]
 [  12  988 1415 ...    0    0    0]
 [3209 1493    5 ...    0    0    0]
 ...
 [7627   20  979 ...    0    0    0]
 [  12  800  359 ...    0    0    0]
 [  12   31   33 ...    0    0    0]], shape=(256, 1485), dtype=int64)
(256, 1485)
3
tf.Tensor(
[[4878 1624    4 ...    0    0    0]
 [  12 3059 5086 ...    0    0    0]
 [ 699 3320  319 ...    0    0    0]
 ...
 [  12  772 2

In [10]:
i[0][0].numpy().shape

(1497,)

In [11]:
i[0].numpy().shape

(256, 1497)

* Not clear for me, when we apply padded_batch, the length is varying then how come it is accepted by NN

# Build & compile model

In [12]:
tokenizer_subwords.vocab_size

8185

In [13]:
EMBED_SIZE=64

In [13]:
model = Sequential([
    Embedding(tokenizer_subwords.vocab_size, EMBED_SIZE),
    Bidirectional(LSTM(64)),
    Dense(64, "relu"),
    Dense(1, "sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [15]:
model.fit(train_dataset, validation_data=test_dataset, epochs=10)

Epoch 1/10
98/98 [==============================] - 52s 438ms/step - loss: 0.6401 - accuracy: 0.6234 - val_loss: 0.4556 - val_accuracy: 0.7892
Epoch 2/10
98/98 [==============================] - 20s 206ms/step - loss: 0.3635 - accuracy: 0.8448 - val_loss: 0.4183 - val_accuracy: 0.8169
Epoch 3/10
98/98 [==============================] - 20s 207ms/step - loss: 0.2842 - accuracy: 0.8870 - val_loss: 0.4472 - val_accuracy: 0.8320
Epoch 4/10
98/98 [==============================] - 21s 213ms/step - loss: 0.2478 - accuracy: 0.9059 - val_loss: 0.4709 - val_accuracy: 0.8378
Epoch 5/10
98/98 [==============================] - 25s 260ms/step - loss: 0.2175 - accuracy: 0.9182 - val_loss: 0.5031 - val_accuracy: 0.8163
Epoch 6/10
98/98 [==============================] - 21s 213ms/step - loss: 0.2051 - accuracy: 0.9225 - val_loss: 0.5988 - val_accuracy: 0.7726
Epoch 7/10
98/98 [==============================] - 21s 211ms/step - loss: 0.1701 - accuracy: 0.9366 - val_loss: 0.5719 - val_accuracy: 0.8070

In [14]:
train_sentences = []
train_labels    = []

test_sentences  = []
test_labels     = []

for row in train:
  train_sentences.append(row[0].numpy())
  train_labels.append(row[1].numpy())


for row in test:
  test_sentences.append(row[0].numpy())
  test_labels.append(row[1].numpy())

In [15]:
train_sentences[0]

array([  62,   18,   41,  604,  927,   65,    3,  644, 7968,   21,   35,
       5096,   36,   11,   43, 2948, 5240,  102,   50,  681, 7862, 1244,
          3, 3266,   29,  122,  640,    2,   26,   14,  279,  438,   35,
         79,  349,  384,   11, 1991,    3,  492,   79,  122,  188,  117,
         33, 4047, 4531,   14,   65, 7968,    8, 1819, 3947,    3,   62,
         27,    9,   41,  577, 5044, 2629, 2552, 7193, 7961, 3642,    3,
         19,  107, 3903,  225,   85,  198,   72,    1, 1512,  738, 2347,
        102, 6245,    8,   85,  308,   79, 6936, 7961,   23, 4981, 8044,
          3, 6429, 7961, 1141, 1335, 1848, 4848,   55, 3601, 4217, 8050,
          2,    5,   59, 3831, 1484, 8040, 7974,  174, 5773,   22, 5240,
        102,   18,  247,   26,    4, 3903, 1612, 3902,  291,   11,    4,
         27,   13,   18, 4092, 4008, 7961,    6,  119,  213, 2774,    3,
         12,  258, 2306,   13,   91,   29,  171,   52,  229,    2, 1245,
       5790,  995, 7968,    8,   52, 2948, 5240, 80

In [16]:
print(type(train_sentences), type(train_sentences[0]))

<class 'list'> <class 'numpy.ndarray'>


In [17]:
train_sentences[1]

array([  12,   31,   93,  867,    7, 1256, 6585, 7961,  421,  365,    2,
         26,   14,    9,  988, 1089,    7,    4, 6728,    6,  276, 5760,
       2587,    2,   81, 6118, 8029,    2,  139, 1892, 7961,    5, 5402,
        246,   25,    1, 1771,  350,    5,  369,   56, 5397,  102,    4,
       2547,    3, 4001,   25,   14, 7822,  209,   12, 3531, 6585, 7961,
         99,    1,   32,   18, 4762,    3,   19,  184, 3223,   18, 5855,
       1045,    3, 4232, 3337,   64, 1347,    5, 1190,    3, 4459,    8,
        614,    7, 3129,    2,   26,   22,   84, 7020,    6,   71,   18,
       4924, 1160,  161,   50, 2265,    3,   12, 3983,    2,   12,  264,
         31, 2545,  261,    6,    1,   66,    2,   26,  131,  393,    1,
       5846,    6,   15,    5,  473,   56,  614,    7, 1470,    6,  116,
        285, 4755, 2088, 7961,  273,  119,  213, 3414, 7961,   23,  332,
       1019,    3,   12, 7667,  505,   14,   32,   44,  208, 7975])

In [18]:
type(train_labels)

list

In [ ]:
train_labels

In [20]:
#train_sentences_fin = np.array(train_sentences)
#test_sentences_fin  = np.array(test_sentences)

#train_labels_fin = np.array(train_labels)
#test_labels_fin  = np.array(test_labels)

In [32]:
from tensorflow.keras.utils import pad_sequences
train_padded = pad_sequences(train_sentences, maxlen=1500, padding="post", truncating="post")
test_padded  = pad_sequences(test_sentences, maxlen=1500, padding="post", truncating="post")

In [33]:
train_padded.shape

(25000, 1500)

In [34]:
len(train_padded[0]), len(train_padded[1])

(1500, 1500)

In [35]:
model = Sequential([
    Embedding(tokenizer_subwords.vocab_size, EMBED_SIZE),
    Bidirectional(LSTM(64)),
    Dense(64, "relu"),
    Dense(1, "sigmoid")
])

model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(train_padded, np.array(train_labels), validation_data=(test_padded, np.array(test_labels)), epochs=10, batch_size=256)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 64)          523840    
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
98/98 [==============================] - 35s 320ms/step - loss: 0.5811 - accuracy: 0.6817 - val_loss: 0